<a href="https://colab.research.google.com/github/ffoxdelorean/ArcGIS_Data_Pipeline/blob/main/CapacityCharge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'delorean-test-1' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()



In [3]:
isone = pd.read_excel('ISONE_capacity_charge_database.xlsx')
miso = pd.read_excel('MISO_capacity_charge_database.xlsx')

In [4]:
!pip install datetime
import datetime as dt
isone['Time_Imported'] = dt.datetime.now()
miso['Time_Imported'] = dt.datetime.now()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.1/247.1 kB 7.7 MB/s eta 0:00:00


In [5]:

isone.capacity_price_dollars_kw_mo = isone.capacity_price_dollars_kw_mo.round(3)
miso.capacity_price_dollars_kw_mo = miso.capacity_price_dollars_kw_mo.round(3)

In [6]:
isone['version'] = isone['version'].astype(str)
miso['version'] = miso['version'].astype(str)

In [17]:
miso['source_region'] = miso['source_region'].astype(str)
miso['auction'] = miso['auction'].astype(str)

In [8]:
import pandas_gbq
schema = [
    {'name': 'index', 'type': 'INTEGER'},
    {'name': 'RTO', 'type': 'STRING'},
    {'name': 'zone', 'type': 'STRING'},
    {'name': 'source_region', 'type': 'STRING'},
    {'name': 'year', 'type': 'INTEGER'},
    {'name': 'month', 'type': 'INTEGER'},
    {'name': 'auction', 'type': 'STRING'},
    {'name': 'capacity_price_dollars_kw_month', 'type': 'FLOAT'},
    {'name': 'version', 'type': 'STRING'},
    {'name': 'source', 'type': 'STRING'},
    {'name': 'Time_Imported', 'type': 'DATETIME'}]

In [14]:
schema_miso = [
    {'name': 'index', 'type': 'INTEGER'},
    {'name': 'RTO', 'type': 'STRING'},
    {'name': 'zone', 'type': 'INTEGER'},
    {'name': 'source_region', 'type': 'STRING'},
    {'name': 'year', 'type': 'INTEGER'},
    {'name': 'month', 'type': 'INTEGER'},
    {'name': 'auction', 'type': 'STRING'},
    {'name': 'capacity_price_dollars_kw_month', 'type': 'FLOAT'},
    {'name': 'version', 'type': 'STRING'},
    {'name': 'source', 'type': 'STRING'},
    {'name': 'Time_Imported', 'type': 'DATETIME'}]

In [18]:
miso.dtypes

RTO                                     object
zone                                     int64
source_region                           object
year                                     int64
month                                    int64
auction                                 object
type                                    object
capacity_price_dollars_kw_mo           float64
version                                 object
source                                  object
Time_Imported                   datetime64[ns]
dtype: object

In [ ]:
pandas_gbq.to_gbq(isone, "markets_iso_ne.isone_capacity_charge", project_id="delorean-test-1", table_schema=schema, if_exists = "replace")


100%|██████████| 1/1 [00:00<00:00, 7839.82it/s]


In [19]:
pandas_gbq.to_gbq(miso, "markets_miso.miso_capacity_charge", project_id="delorean-test-1", table_schema=schema_miso, if_exists = "replace")


100%|██████████| 1/1 [00:00<00:00, 6647.07it/s]
